# Initialize the bert model

In [1]:
from transformers import TFAutoModel

In [2]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Metal device set to: Apple M1


2022-11-14 19:12:33.662072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-14 19:12:33.662438: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identica

In [4]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [5]:
import tensorflow as tf

In [11]:
#two inputs
input_ids = tf.keras.layers.Input(shape=(512,), name ='input_ids',
                                 dtype = 'int32')
mask = tf.keras.layers.Input(shape=(512,), name ='attention_mask',
                                 dtype = 'int32')

#transformer
embeddings = bert.bert(input_ids, attention_mask=mask)[1]

#classifier head
x = tf.keras.layers.Dense(1024, activation = 'relu')(embeddings)
y = tf.keras.layers.Dense(5, activation = 'softmax', name='outputs')(x)

In [12]:
model = tf.keras.Model(inputs =[input_ids,mask],outputs = y)

In [14]:
model.layers[2].trainable = False

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [18]:
optimizer = tf.keras.optimizers.Adam(lr=5e-5,decay =1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer,loss=loss,metrics=[acc])

/Users/kaspar/tensorflow-transformers/env/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [20]:
element_spec=({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 
            'attention_mask': tf.TensorSpec(shape=(16, 512),
              dtype=tf.int64, name=None)},
              tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))



In [21]:
train_ds = tf.data.experimental.load('train',element_spec=element_spec)
val_ds = tf.data.experimental.load('val',element_spec=element_spec)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


In [23]:
train_ds.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [ ]:
history = model.fit(train_ds,validation_data = val_ds,epochs =3)